In [1]:
import pandas as pd
import requests
import json
import datetime
import concurrent.futures
import swifter
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from numpy import random

/opt/spark/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
if os.path.isfile('dados/TJRN_4º_JUIZADO_ESPECIAL_DA_FAZENDA_PÚBLICA_2018-01-01.csv'):
    df_tjrn = pd.read_csv('dados/TJRN_4º_JUIZADO_ESPECIAL_DA_FAZENDA_PÚBLICA_2018-01-01.csv', header='infer', sep=';', compression='zip')
else:
    df_tjrn = pd.read_csv(glob.glob("dados/TJRN*.csv")[0], header='infer', sep=';', compression='zip')
    print(f'Carregando o dataset referente ao {glob.glob("dados/TJRN*.csv")[0]}')
    

Carregando o dataset referente ao dados/TJRN_1º_JUIZADO_ESPECIAL_DA_FAZENDA_PÚBLICA_2018-01-01.csv


In [4]:
df_tjrn.drop_duplicates(inplace=True)
if 'sort' in df_tjrn.columns:
    df_tjrn.drop(columns=['sort'], inplace=True)
if 'classe' in df_tjrn.columns:
    df_tjrn.drop(columns=['classe'], inplace=True)    
if 'grau' in df_tjrn.columns:
    df_tjrn.drop(columns=['grau'], inplace=True)
if 'orgao_julgador' in df_tjrn.columns:
    df_tjrn.drop(columns=['orgao_julgador'], inplace=True)
    
if df_tjrn['assuntos'].isna().sum() > (0.8 * len(df_tjrn['assuntos'])):
    df_tjrn.drop(columns=['assuntos'], inplace=True)    

In [5]:
df_tjrn.head(1)

,numero_processo,data_ajuizamento,ultima_atualizacao,codigo,municipio,assuntos,movimentos
0,8533934020198205001,2019-11-11 08:12:17-03:00,2022-10-01 00:48:13.370000-03:00,17460,3201,"['', '']","[[26, 'Distribuição', Timestamp('2019-11-10 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2019-11-10 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2019-12-20 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2019-12-20 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-01-08 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-03-24 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2020-03-29 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2020-04-05 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-05-17 21:00:00-0300', tz='America/Sao_Paulo')], [581, 'Documento', Timestamp('2020-06-17 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2020-06-17 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-07-06 21:00:00-0300', tz='America/Sao_Paulo')], [123, 'Remessa', Timestamp('2020-07-12 21:00:00-0300', tz='America/Sao_Paulo')], [11010, 'Mero expediente', Timestamp('2019-11-11 21:00:00-0300', tz='America/Sao_Paulo')], [471, 'Pronúncia de Decadência ou Prescrição', Timestamp('2020-04-05 21:00:00-0300', tz='America/Sao_Paulo')]]"


In [6]:

df_tjrn['data_ajuizamento'] = pd.to_datetime(df_tjrn['data_ajuizamento'], utc=True)
print(df_tjrn.info())
df_tjrn['data_ajuizamento'] = df_tjrn['data_ajuizamento'].dt.strftime('%Y-%m-%d')
df_tjrn['ultima_atualizacao'] = pd.to_datetime(df_tjrn['ultima_atualizacao'], format='mixed').dt.strftime('%Y-%m-%d')
print(df_tjrn.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23634 entries, 0 to 23633
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   numero_processo     23634 non-null  int64              
 1   data_ajuizamento    23634 non-null  datetime64[ns, UTC]
 2   ultima_atualizacao  23634 non-null  object             
 3   codigo              23634 non-null  int64              
 4   municipio           23634 non-null  int64              
 5   assuntos            23634 non-null  object             
 6   movimentos          23634 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(3), object(3)
memory usage: 1.3+ MB
None
       numero_processo data_ajuizamento ultima_atualizacao  codigo  municipio  \
0  8533934020198205001       2019-11-11         2022-10-01   17460       3201   
1  8223265220228205001       2022-04-11         2022-10-01   17460       3201   

   assuntos  \
0  ['

In [7]:
movimentacoes = list(df_tjrn.movimentos.values)
print(len(movimentacoes))

23634


In [8]:
print(movimentacoes[0])
print(df_tjrn['movimentos'][0])

[[26, 'Distribuição', Timestamp('2019-11-10 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2019-11-10 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2019-12-20 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2019-12-20 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-01-08 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-03-24 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2020-03-29 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2020-04-05 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-05-17 21:00:00-0300', tz='America/Sao_Paulo')], [581, 'Documento', Timestamp('2020-06-17 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2020-06-17 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-07-06 21:00:00-

In [9]:
""" lista_movimentacoes_completa = []
lista_codigos = list()
lista_nomes = list()
lista_data = list()
i = 0

for movimento in movimentacoes:
    if '272' in movimento.split():
        print(movimento)
        continue 
        #print(len(list(movimento.split(',')))/4)
    else:
        if len(list(movimento.split(','))) % 4 == 0:
            for elemento in range(0,int(len(list(movimento.split(',')))/4),4):
                if elemento != None:
                    codigo = movimento.split(',')[elemento].replace('[','').replace(' ', '')
                    nome_movimentacao = movimento.split(',')[elemento+1].replace('[','').replace(' ', '')
                    data_movimentacao = movimento.split(',')[elemento+2].replace('[','').replace('Timestamp(','').split(' ')[1].replace(' ', '').replace('\'', '')
                    lista_temp = list()
                    lista_temp.append([codigo, nome_movimentacao, data_movimentacao])
                    #lista_temp.append(nome_movimentacao)
                    #lista_temp.append(data_movimentacao)
                    #lista_movimentacoes_completa.append([])
                    lista_movimentacoes_completa.append(lista_temp)
                else:
                    lista_temp = [-1, -1, -1]
                    #lista_movimentacoes_completa.append([])
                    lista_movimentacoes_completa.append(lista_temp)
            i = i + 1 """

" lista_movimentacoes_completa = []\nlista_codigos = list()\nlista_nomes = list()\nlista_data = list()\ni = 0\n\nfor movimento in movimentacoes:\n    if '272' in movimento.split():\n        print(movimento)\n        continue \n        #print(len(list(movimento.split(',')))/4)\n    else:\n        if len(list(movimento.split(','))) % 4 == 0:\n            for elemento in range(0,int(len(list(movimento.split(',')))/4),4):\n                if elemento != None:\n                    codigo = movimento.split(',')[elemento].replace('[','').replace(' ', '')\n                    nome_movimentacao = movimento.split(',')[elemento+1].replace('[','').replace(' ', '')\n                    data_movimentacao = movimento.split(',')[elemento+2].replace('[','').replace('Timestamp(','').split(' ')[1].replace(' ', '').replace(''', '')\n                    lista_temp = list()\n                    lista_temp.append([codigo, nome_movimentacao, data_movimentacao])\n                    #lista_temp.append(nome_mov

In [10]:
lista_movimentacoes_completa = []
lista_codigos = list()
lista_nomes = list()
lista_data = list()
i = 0

for movimento in movimentacoes:
    if '272' in movimento.split():
        print(movimento)
        #continue 
        #print(len(list(movimento.split(',')))/4)
    else:
        if len(list(movimento.split(','))) % 4 == 0:
            lista_temp = list()
            for elemento in range(0,int(len(list(movimento.split(',')))/4),4):
                codigo = movimento.split(',')[elemento].replace('[','').replace(' ', '')
                nome_movimentacao = movimento.split(',')[elemento+1].replace('[','').replace(' ', '')
                data_movimentacao = movimento.split(',')[elemento+2].replace('[','').replace('Timestamp(','').split(' ')[1].replace(' ', '').replace('\'', '')
                lista_temp.append([codigo, nome_movimentacao, data_movimentacao])
        else:
            lista_temp.append([-1,-1,-1])
    lista_movimentacoes_completa.append(lista_temp)


In [11]:
print(len(lista_movimentacoes_completa))
#lista_movimentacoes_completa = list(filter(None, lista_movimentacoes_completa))
print(len(movimentacoes), '\t', movimentacoes[0])
print(len(lista_movimentacoes_completa), '\t', lista_movimentacoes_completa[0])


23634
23634 	 [[26, 'Distribuição', Timestamp('2019-11-10 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2019-11-10 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2019-12-20 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2019-12-20 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-01-08 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-03-24 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2020-03-29 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2020-04-05 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-05-17 21:00:00-0300', tz='America/Sao_Paulo')], [581, 'Documento', Timestamp('2020-06-17 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2020-06-17 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-0

In [12]:
print(type(lista_movimentacoes_completa), len(lista_movimentacoes_completa))
print(type(lista_movimentacoes_completa[0]), len(lista_movimentacoes_completa[2]))
print(lista_movimentacoes_completa[5200])
print(lista_movimentacoes_completa[:10])

<class 'list'> 23634
<class 'list'> 4
[['581', "'Documento'", '2022-05-09'], ['581', "'Documento'", '2022-04-17'], ['60', "'Expediçãodedocumento'", '2022-01-26'], ['581', "'Documento'", '2022-01-26'], ['581', "'Documento'", '2022-01-26'], ['60', "'Expediçãodedocumento'", '2021-11-22'], ['85', "'Petição'", '2021-10-31']]
[[['26', "'Distribuição'", '2019-11-10'], ['51', "'Conclusão'", '2019-11-10'], ['60', "'Expediçãodedocumento'", '2019-12-20'], ['60', "'Expediçãodedocumento'", '2019-12-20']], [['26', "'Distribuição'", '2022-04-10'], ['51', "'Conclusão'", '2022-04-10'], ['60', "'Expediçãodedocumento'", '2022-05-05'], ['60', "'Expediçãodedocumento'", '2022-05-05']], [['26', "'Distribuição'", '2019-08-13'], ['51', "'Conclusão'", '2019-08-13'], ['60', "'Expediçãodedocumento'", '2019-08-13'], ['85', "'Petição'", '2019-10-13']], [['26', "'Distribuição'", '2020-06-24'], ['51', "'Conclusão'", '2020-06-24'], ['60', "'Expediçãodedocumento'", '2020-06-27'], ['60', "'Expediçãodedocumento'", '2020-

In [13]:
print(lista_movimentacoes_completa[0])
print(df_tjrn['movimentos'][0])

[['26', "'Distribuição'", '2019-11-10'], ['51', "'Conclusão'", '2019-11-10'], ['60', "'Expediçãodedocumento'", '2019-12-20'], ['60', "'Expediçãodedocumento'", '2019-12-20']]
[[26, 'Distribuição', Timestamp('2019-11-10 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2019-11-10 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2019-12-20 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2019-12-20 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-01-08 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-03-24 21:00:00-0300', tz='America/Sao_Paulo')], [51, 'Conclusão', Timestamp('2020-03-29 21:00:00-0300', tz='America/Sao_Paulo')], [60, 'Expedição de documento', Timestamp('2020-04-05 21:00:00-0300', tz='America/Sao_Paulo')], [85, 'Petição', Timestamp('2020-05-17 21:00:00-0300', tz='America/Sao_Paulo')], [581, 'Documento', Timestamp('2020-06-17 21:00:00-0

## Estatísticas

In [14]:
media_movimentacoes = 0.0
stdDev_movimentacoes = 0.0
soma_movimentacoes = 0.0
duracao_media_movimentacoes = 0.0
numero_movimentacoes = 0
soma_duracao_movimentacoes = datetime
dicionario_distribuicao_numero_mov = dict()

for i in lista_movimentacoes_completa:
    #print(i)
    if str(len(i)) not in dicionario_distribuicao_numero_mov:
        dicionario_distribuicao_numero_mov[str(len(i))] = []
    dicionario_distribuicao_numero_mov[str(len(i))].append(i)

for i in range(0,len(lista_movimentacoes_completa),1):   
    soma_movimentacoes += len(lista_movimentacoes_completa[i])
    #print(soma_movimentacoes, '\t', i)

for i in range(0,len(lista_movimentacoes_completa),1):
    for j in range(0,(len(lista_movimentacoes_completa[i])-1),4):
        if len(lista_movimentacoes_completa[i][j]) > 4:          
            print(lista_movimentacoes_completa[i][j+4][2] - lista_movimentacoes_completa[i][j][2])
        #print(lista_movimentacoes_completa[i][j])
    print('')

media_movimentacoes = soma_movimentacoes/len(lista_movimentacoes_completa)

print(f'Média das movimentações: {media_movimentacoes}')
    


In [15]:
movimentacoes_gabinete = pd.read_csv('dados/movimentos_gabinete.csv', sep=';', header='infer')
movimentacoes_gabinete['codigo'] = movimentacoes_gabinete['vazio.1']
movimentacoes_gabinete.drop(columns=['complemento','vazio', 'vazio.2', 'vazio.1'], inplace=True)
lista_movimentacoes_sentenca = movimentacoes_gabinete[movimentacoes_gabinete['nome'] == 'Julgamento']['codigo'].to_list()
print(movimentacoes_gabinete.head(5))

     quem_faz                   nome  \
0  Magistrado  Cooperação Judiciária   
1  Magistrado                Decisão   
2  Magistrado                Decisão   
3  Magistrado                Decisão   
4  Magistrado                Decisão   

                                  detalhamento  codigo  
0                                          NaN   15185  
1                                          NaN       3  
2        Acolhimento de Embargos de Declaração   15162  
3                       Acolhimento de exceção     133  
4  Acolhimento de exceção de pré-executividade     335  


In [16]:
print(movimentacoes_gabinete[movimentacoes_gabinete['nome'] == 'Julgamento'].head(5))

       quem_faz        nome  \
149  Magistrado  Julgamento   
150  Magistrado  Julgamento   
151  Magistrado  Julgamento   
152  Magistrado  Julgamento   
153  Magistrado  Julgamento   

                                                                detalhamento  \
149                                                                      NaN   
150                                                  Com Resolução do Mérito   
151           Com Resolução do Mérito Acolhimento de Embargos de Declaração    
152  Com Resolução do Mérito Acolhimento em parte de Embargos de Declaração    
153                                        Com Resolução do Mérito Consulta    

     codigo  
149     193  
150     385  
151     198  
152     871  
153   12326  


In [17]:
movimentacoes_secretaria = pd.read_csv('dados/movimentos_secretaria.csv', sep=';', header='infer')
print(movimentacoes_secretaria.head(5))
movimentacoes_secretaria.drop(columns=['Unnamed: 3','complemento'], inplace=True)
lista_movimentacoes_secretaria = movimentacoes_secretaria['codigo'].to_list()
print(movimentacoes_secretaria.head(5))

       quem_faz                                                nome  \
0  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
1  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
2  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
3  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
4  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   

              detalhamento  Unnamed: 3 complemento  codigo  
0      Ações de secretário         NaN         NaN      48  
1              Apensamento         NaN         NaN     135  
2             Arquivamento         NaN         NaN     861  
3  Arquivamento Definitivo  Definitivo         NaN     246  
4  Arquivamento Provisório  Provisório         NaN     245  
       quem_faz                                                nome  \
0  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídico   
1  Serventuário  Escrivão/Diretor de Secretaria/Secretário Jurídi

In [18]:
def calcular_data_sentenca(movimentacoes):
    processos_sentenciados = 0
    processos_nao_sentenciados = 0    
    lista_data_sentenca = list()
    for i in range(0,(len(movimentacoes)),1):
        processado = False
        if len(movimentacoes[i]) > 3:
            for j in range(0,(len(movimentacoes[i])-1),3):
                #print(movimentacoes[i][j][0])
                if processado:
                   break 
                if int(movimentacoes[i][j][0]) in lista_movimentacoes_sentenca:
                    data = pd.to_datetime(movimentacoes[i][j][2])
                    data = data.date().strftime("%Y-%m-%d")
                    lista_data_sentenca.append(data)
                    processos_sentenciados = processos_sentenciados + 1
                    processado = True
                    #print(f'Processando: {i}: Sentença encontrada')
            if not processado:
                lista_data_sentenca.append('-1')
                processado = True
                processos_nao_sentenciados = processos_nao_sentenciados + 1         
   
                #print(f'data adicionada: {data}')    
        else:
            #print(movimentacoes[i])
            lista_data_sentenca.append('-1')
            processos_nao_sentenciados = processos_nao_sentenciados + 1
            #print(f'data adicionada: {data_default_nao_sentenciados}')
            #print(f'Processando: {i}: SEM movimentação do gabinete')
    print(f'processos sentenciados: {processos_sentenciados}\tprocessos não sentenciados: {processos_nao_sentenciados}')
    return lista_data_sentenca

In [19]:
#print(calcular_data_sentenca(lista_movimentacoes_completa))
print(len(lista_movimentacoes_completa), len(calcular_data_sentenca(lista_movimentacoes_completa)))

processos sentenciados: 503	processos não sentenciados: 23131
23634 23634


In [20]:
# funciona ok.
def calcular_tempo_entre_movimentacoes1e2(movimentacoes):
    lista_duracao = list()
    for i in range(0,(len(movimentacoes) -1),1):
        if len(movimentacoes[i]) > 3:
            #print(type(movimentacoes[i][0][2]), '\t', movimentacoes[i][0][2])
            #print(type(movimentacoes[i+1][0][2]), '\t', movimentacoes[i+1][0][2])
            duracao = abs(pd.to_datetime(movimentacoes[i][1][2]) - pd.to_datetime(movimentacoes[i][0][2]))/np.timedelta64(1, 'D')
            lista_duracao.append(duracao)
            #if duracao > 60:
                #print(duracao, '\t', movimentacoes[i][1][2], ' - ', movimentacoes[i][0][2], '\t', movimentacoes[i])                  

        else: 
            lista_duracao.append(' ')
            
    return lista_duracao        

In [21]:
percentual_sentenciados = float("{:.4f}".format((len(calcular_data_sentenca(lista_movimentacoes_completa))/len(lista_movimentacoes_completa))))
print(f'Percentual da base de casos sentenciados: {percentual_sentenciados}%')
#print(len(calcular_tempo_entre_movimentacoes1e2(lista_movimentacoes_completa)))

processos sentenciados: 503	processos não sentenciados: 23131
Percentual da base de casos sentenciados: 1.0%


In [22]:
df_tjrn['data_sentenca'] = calcular_data_sentenca(lista_movimentacoes_completa)

processos sentenciados: 503	processos não sentenciados: 23131


In [23]:
df_sentenciados = df_tjrn[df_tjrn['data_sentenca'] != '-1']
df_nao_sentenciados = df_tjrn[df_tjrn['data_sentenca'] == '-1']

In [29]:
print(df_sentenciados.shape)
print(df_nao_sentenciados.shape)
print(df_tjrn.shape)

(503, 8)
(23131, 8)
(23634, 8)


In [27]:
print(df_sentenciados.head(1))

           numero_processo data_ajuizamento ultima_atualizacao  codigo  \
11364  8146554620208205001       2020-04-25         2023-10-16   17460   

       municipio                              assuntos  \
11364       3201  ['Causas Supervenientes à Sentença']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [31]:
print(df_sentenciados['data_ajuizamento'].value_counts())

data_ajuizamento
2019-09-25    6
2020-06-16    5
2020-08-16    4
2020-08-13    4
2019-11-18    4
2019-08-26    4
2020-07-07    4
2022-01-12    4
2022-02-25    3
2020-07-14    3
2020-08-07    3
2020-09-24    3
2021-08-30    3
2020-08-11    3
2022-03-10    3
2020-08-27    3
2021-12-31    3
2019-09-23    3
2020-09-04    3
2022-04-13    3
2022-02-17    3
2019-09-03    3
2022-04-28    3
2020-05-18    3
2022-07-29    3
2022-12-06    3
2020-07-15    3
2018-08-07    3
2019-05-27    3
2019-07-25    3
2020-06-05    2
2020-04-06    2
2020-06-30    2
2020-09-10    2
2022-04-02    2
2020-07-06    2
2020-06-19    2
2018-08-23    2
2020-12-01    2
2019-05-02    2
2020-02-19    2
2021-02-25    2
2020-09-14    2
2022-06-18    2
2022-08-05    2
2020-10-13    2
2021-09-09    2
2020-09-02    2
2019-09-04    2
2019-07-24    2
2018-06-20    2
2020-07-29    2
2022-03-14    2
2022-11-04    2
2020-06-22    2
2021-03-08    2
2019-04-02    2
2020-06-14    2
2020-02-02    2
2022-01-26    2
2019-01-10    2
2020-10